# CENG403 - Spring 2024 - THE1

In this take-home-exam, you will implement your own tensor library, called CerGen (short for CENG Gergen -- gergen: one of the Turkish translations of the term tensor).

Example usage:

```python
from cergen import rastgele_gercek,rastgele_dogal,cekirdek, gergen

boyut = (1,)
aralik = (0, 10)
g0 = rastgele_gercek(boyut, aralik)
print(g0)
1 boyutlu gergen:
8

g1 = gergen([[1, 2, 3], [4, 5, 6]])
print(g1)
2x3 boyutlu gergen:
[[1, 2, 3]
 [4, 5, 6]]

g2 = gergen(rastgele_dogal((3, 1)))
print(g2)
3x1 boyutlu gergen
[[6],
[5],
[2]]

print((g1 * g2))
2x1 boyutlu gergen:
[[22],
[61]]

g3 = (g1 * (g2 + 3)).topla()

```


## 1 Task Description
In this homework, we introduce the gergen class, a custom data structure designed to provide a
hands-on experience with fundamental array operations, mirroring some functionalities typically
found in libraries like NumPy.

## Fundamental Operations:
Random number generation:

In [ ]:
import random

def cekirdek(sayi: int):
    random.seed(sayi)

def empty_matrix(shape, element = 0):
        if len(shape) == 1:
            return [element] * shape[0]
        else:
            return [empty_matrix(shape[1:], element) for _ in range(shape[0])]

def fill_list(length, dagilim, aralik, format):
    if (format == 'dogal'):
        if(dagilim == 'uniform'):
            return [random.randint(aralik[0], aralik[1]) for _ in range(length)]

    if (format == 'gercek'):
        if(dagilim == 'uniform'):
            return [random.uniform(aralik[0], aralik[1]) for _ in range(length)]


def rastgele_helper(boyut, aralik, dagilim, format):
    if (len(boyut) < 1):
        return None

    result = []

    if (len(boyut) == 2) :
        res = []
        for _ in range(boyut[0]):
            res.append(fill_list(boyut[1], dagilim, aralik, format))
        return res

    if (len(boyut) == 1) :
        return fill_list(boyut[0], dagilim, aralik, format)

    for i in range(boyut[0]):
        result.append(rastgele_helper(boyut[1:], aralik, dagilim, format)[:])

    return result


def rastgele_dogal(boyut, aralik=(0,100), dagilim='uniform'):

    if (dagilim != 'uniform'):
        raise ValueError()

    veri = rastgele_helper(boyut, aralik, dagilim, 'dogal')

    return gergen(veri)


def rastgele_gercek(boyut, aralik=(0.0, 1.0), dagilim='uniform'):

    if (dagilim != 'uniform'):
        raise ValueError()

    veri = rastgele_helper(boyut, aralik, dagilim, 'gercek')

    return gergen(veri)



Operation class implementation:

In [ ]:
class Operation:
    def __call__(self, *operands):
        """
        Makes an instance of the Operation class callable.
        Stores operands and initializes outputs to None.
        Invokes the forward pass of the operation with given operands.

        Parameters:
            *operands: Variable length operand list.

        Returns:
            The result of the forward pass of the operation.
        """
        self.operands = operands
        self.outputs = None
        return self.ileri(*operands)

    def ileri(self, *operands):
        """
        Defines the forward pass of the operation.
        Must be implemented by subclasses to perform the actual operation.

        Parameters:
            *operands: Variable length operand list.

        Raises:
            NotImplementedError: If not overridden in a subclass.
        """
        raise NotImplementedError


In [ ]:
import math
from typing import Union

class gergen:

    __veri = None
    D = None
    __boyut = None

    def __init__(self, veri=None):
        if (veri is None):
            return

        if (isinstance(veri, int) or isinstance(veri, float)):
            self.__veri = veri
            self.__boyut = ()
            self.D = veri
            return

        self.__veri = veri
        temp = []
        iterator = veri

        if (iterator == []): return

        while(isinstance(iterator,list)):
            temp.append(len(iterator))
            iterator = iterator[0]

        self.__boyut = tuple(temp)

        self.D = self.transposeFinder()


    def indexIncrement(self, indexList):

        tdims = self.__boyut[::-1]
        carry = 1
        index = 0
        while (carry > 0):
            indexList[index] +=1
            carry = 0
            if(indexList[index] == tdims[index]):
                indexList[index] = 0
                index +=1
                carry = 1

        return indexList



    def transposeFinder(self):

        def getIndex(indexList):
            res = 0
            tDims = self.__boyut[::-1]
            for i in range(len(indexList)):
                res += indexList[i] * math.prod(tDims[i+1:])
            return res

        flatData = self.flatten(self.__veri)
        transpose = [0] * len(flatData)
        indexList = [0]* len(self.__boyut)

        for i in range(len(transpose) - 1):
            indexOfElement = getIndex(indexList)
            transpose[indexOfElement] = flatData[i]
            self.indexIncrement(indexList)

        transpose[len(flatData)-1] = flatData[len(flatData)-1]

        transpose = self.divide(transpose, self.__boyut[::-1])

        return transpose


    def __getitem__(self, index):
        return self.__veri[index]

    def strHelper(self, veri, depth):

        if(not isinstance(veri[0],list)):
            return str(veri)

        result = "["
        for vericik in veri[:-1]:
            result += self.strHelper(vericik, depth + 1) + ('\n' * (len(self.__boyut) - depth - 1))

        result += self.strHelper(veri[-1], depth + 1)

        result += ']'

        return result

    def __str__(self):
        result = ""

        if (self.__veri is None):
            return "0 boyutlu gergen:"

        if (isinstance(self.__veri, Union[int,float])):
            return "0 boyutlu skaler gergen:" + '\n' + str(self.__veri)

        for dim in self.__boyut:
            result += str(dim) + 'x'

        result = (result[:-1] + " boyutlu gergen:" + '\n')

        result += self.strHelper(self.__veri, 0)

        return result

    def recursiveOperatorHelper(self, selfVeri, otherVeri, function):
        if (not isinstance(selfVeri, list)): raise TypeError()

        if (not isinstance(selfVeri[0], list)):
            return [function(selfVeri[i], otherVeri[i]) for i in range(len(selfVeri))]

        result = []
        for i in range(len(selfVeri)):
            result.append(self.recursiveOperatorHelper(selfVeri[i], otherVeri[i], function))

        return result



    def __rmul__(self, other):
        return self * other

    def __mul__(self, other: Union['gergen', int, float]) -> 'gergen':

        if (isinstance(self.__veri, Union[int,float])):
            value = self.__veri * other
            return gergen(value) if not isinstance(value, gergen) else value

        if( isinstance(other, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x*other)))

        if(isinstance(other.__veri, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x*other.__veri)))

        if(other.__boyut != self.__boyut): raise ValueError()

        return gergen(self.recursiveOperatorHelper(self.__veri, other.__veri, (lambda x,y : x*y)))



    def __rtruediv__(self,other):
        if (self.__boyut != ()):
            scalar = empty_matrix(self.__boyut, other)
            return gergen(scalar) / self
        else:
            return gergen(other / self.__veri)


    def __truediv__(self, other: Union['gergen', int, float]) -> 'gergen':

        if (isinstance(self.__veri, Union[int,float])):
            value = self.__veri / other
            return gergen(value) if not isinstance(value, gergen) else value

        if( isinstance(other, Union[int,float])):
            if (other == 0): raise ZeroDivisionError()
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x/other)))

        if(isinstance(other.__veri, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x / other.__veri)))

        if(other.__boyut != self.__boyut): raise ValueError()

        return gergen(self.recursiveOperatorHelper(self.__veri, other.__veri, (lambda x,y : x/y)))



    def __radd__(self, other):
        return self + other

    def __add__(self, other: Union['gergen', int, float]) -> 'gergen':

        if (isinstance(self.__veri, Union[int,float])):
            value = self.__veri + other
            return gergen(value) if not isinstance(value, gergen) else value

        if(isinstance(other, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x+other)))

        if(isinstance(other.__veri, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x+other.__veri)))


        if(other.__boyut != self.__boyut): raise ValueError()

        return gergen(self.recursiveOperatorHelper(self.__veri, other.__veri, (lambda x,y : x+y)))



    def __rsub__(self,other):
        if (self.__boyut != ()):
            scalar = empty_matrix(self.__boyut, other)
            return gergen(scalar) - self
        else:
            return gergen(other - self.__veri)



    def __sub__(self, other: Union['gergen', int, float]) -> 'gergen':

        if (isinstance(self.__veri, Union[int,float])):
            value = self.__veri - other
            return gergen(value) if not isinstance(value, gergen) else value

        if( isinstance(other, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x-other)))

        if(isinstance(other.__veri, Union[int,float])):
            return gergen(self.general_recursive_helper(self.__veri, (lambda x: x-other.__veri)))

        if(other.__boyut != self.__boyut): raise ValueError()

        return gergen(self.recursiveOperatorHelper(self.__veri, other.__veri, (lambda x,y : x-y)))

    def uzunluk(self):
        if (self.__boyut == ()): return 1

        return math.prod(self.__boyut)

    def boyut(self):
        return self.__boyut

    def devrik(self):
        return gergen(self.D)

    def general_recursive_helper(self, veri, function):
        if (not isinstance(veri, list)):
            return function(veri)

        if (not isinstance(veri[0],list)):
            return [function(x) for x in veri]

        result = []
        for vericik in veri:
            result.append(self.general_recursive_helper(vericik, function))

        return result

    def cumulative_recursive_helper(self, veri, function):
        if (not isinstance(veri, list)):
            return function(veri)

        accumulator = 0

        if (not isinstance(veri[0],list)):
            for vericik in veri:
                accumulator += function(vericik)
            return accumulator

        for vericik in veri:
            accumulator += self.cumulative_recursive_helper(vericik, function)

        return accumulator


    def sin(self):
        return gergen(self.general_recursive_helper(self.__veri, math.sin))

    def cos(self):
        return gergen(self.general_recursive_helper(self.__veri, math.cos))

    def tan(self):
        return gergen(self.general_recursive_helper(self.__veri, math.tan))

    def us(self, n: int):
        return gergen(self.general_recursive_helper(self.__veri, (lambda x : x**n)))

    def log(self):
        return gergen(self.general_recursive_helper(self.__veri, (lambda x : math.log(x,10))))

    def ln(self):
        return gergen(self.general_recursive_helper(self.__veri, math.log))

    def L1(self):
        return self.cumulative_recursive_helper(self.__veri, abs)

    def L2(self):
        return math.sqrt(self.cumulative_recursive_helper(self.__veri, (lambda x: x**2)))

    def Lp(self, p):
        if (p <= 0): raise ValueError("p should be positive.")
        return (self.cumulative_recursive_helper(self.__veri, (lambda x: abs(x)**p)) ** (1/p))

    def listeye(self):
        return self.__veri

    def flatten(self, veri):
        res = []
        if (isinstance(veri,list)):
            if (not isinstance(veri[0],list)):
                return veri
            for vericik in veri:
                res += self.flatten(vericik)
            return res
        return veri


    def duzlestir(self):
        return gergen(self.flatten(self.__veri))


    def divide(self, veri, dims):

        if(len(veri) != math.prod(dims)): raise ValueError()
        if(len(dims) < 1): raise ValueError()
        if(len(dims) == 1): return veri

        step = math.prod(dims[1:])

        start = 0
        end = step
        res = []

        for i in range(dims[0]):
            res.append(self.divide(veri[start:end],dims[1:]))
            start += step
            end += step
        return res


    def boyutlandir(self, yeni_boyut):
        veri = self.flatten(self.__veri)
        return gergen(self.divide(veri,yeni_boyut))


    def ic_carpim(self, other):
        if(not isinstance(other, gergen)): raise TypeError()

        if(len(self.__boyut) == 1):
            if (len(other.__boyut != 1)): raise ValueError()
            if (self.__boyut != other.__boyut): raise ValueError()

            return sum([self.__veri[i]*other.__veri[i] for i in range(self.__boyut[0])])

        if(len(self.__boyut) != 2): raise ValueError()
        if(len(other.__boyut) != 2): raise ValueError()

        if(self.__boyut[1] != other.__boyut[0]): raise ValueError()


        return gergen([[sum([self_vericik[i]*ot_vericik[i] for i in range(self.__boyut[1])]) for ot_vericik in other.D] for self_vericik in self.__veri])




    def dis_carpim(self, other):
        if(not isinstance(other, gergen)): raise TypeError()

        if(not len(self.__boyut) == 1): raise ValueError()
        if(not len(other.__boyut) == 1): raise ValueError()

        return gergen([[vericik_x * vericik_y for vericik_y in other.__veri] for vericik_x in self.__veri])

    def toplaHelper(self, veri, eksen, derinlik, ortalama = False):
        if (derinlik == eksen):
            stepSize = math.prod(self.__boyut[(eksen + 1):])

            divider = len(veri) / stepSize
            if (ortalama): return [sum(veri[i::stepSize])/divider for i in range(stepSize)]

            return [sum(veri[i::stepSize]) for i in range(stepSize)]

        else:
            res = []
            stepSize = len(veri) // (self.__boyut[derinlik])
            for i in range(self.__boyut[derinlik]):
                res += self.toplaHelper(veri[(stepSize * i):(stepSize * i + stepSize)], eksen, derinlik+1, ortalama)
            return res

    def topla(self, eksen=None):
        if eksen is None:
            return self.cumulative_recursive_helper(self.__veri, (lambda x : x))

        if (not isinstance(eksen, Union[int,float])): raise TypeError()

        if (eksen > len(self.__boyut)): raise ValueError()

        flatData = self.flatten(self.__veri)

        unshaped = self.toplaHelper(flatData, eksen, 0)

        shaped = self.divide(unshaped, (self.__boyut[0:eksen] + self.__boyut[eksen+1:]))

        return gergen(shaped)

    def ortalama(self, eksen=None):
        if eksen is None:
          return self.topla() / self.uzunluk()

        if (eksen > len(self.__boyut)): raise ValueError()
        if (not isinstance(eksen, Union[int,float])): raise TypeError()


        flatData = self.flatten(self.__veri)

        unshaped = self.toplaHelper(flatData, eksen, 0, True)

        shaped = self.divide(unshaped, (self.__boyut[0:eksen] + self.__boyut[eksen+1:]))

        return gergen(shaped)

In [ ]:
a = rastgele_dogal((4,2),(1,4))
b = rastgele_dogal((2,4),(1,4))

print(a)
print()
print(b)
print()
print(a.ic_carpim(b))


4x2 boyutlu gergen:
[[3, 1]
[1, 2]
[2, 3]
[2, 3]]

2x4 boyutlu gergen:
[[2, 1, 3, 1]
[1, 3, 3, 1]]

4x4 boyutlu gergen:
[[7, 6, 12, 4]
[4, 7, 9, 3]
[7, 11, 15, 5]
[7, 11, 15, 5]]


## 2 Compare with NumPy

In [ ]:
import numpy as np              # NumPy, for working with arrays/tensors
import time                     # For measuring time

**Example 1:**
Using rastgele_gercek(), generate two gergen objects with shapes (64,64) and calculate the a.ic carpim(b). Then, calculate the same function with NumPy and report the time and difference.

In [ ]:
def example_1():
    boyut = (64,64)
    g1 = rastgele_gercek(boyut)
    g2 = rastgele_gercek(boyut)

    start = time.time()

    birinci = g1.ic_carpim(g2)

    end = time.time()


    np_g1 = np.array(g1.listeye())
    np_g2 = np.array(g2.listeye())

    start_np = time.time()

    ikinci = np.dot(np_g1, np_g2)

    end_np = time.time()

    birinci = np.array(birinci.listeye())

    print("Time taken for gergen:", end-start)
    print("Time taken for numpy:", end_np-start_np)
    print("Difference:", birinci- ikinci)

    """
    Time taken for gergen: 0.031882524490356445
    Time taken for numpy: 9.512901306152344e-05
    Difference: [[ 0.00000000e+00  0.00000000e+00  3.55271368e-15 ...  0.00000000e+00
      0.00000000e+00 -1.77635684e-15]
    [ 0.00000000e+00 -3.55271368e-15  0.00000000e+00 ...  0.00000000e+00
      0.00000000e+00  1.77635684e-15]
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.55271368e-15
      -3.55271368e-15 -3.55271368e-15]
    ...
    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
      0.00000000e+00 -1.77635684e-15]
    [ 3.55271368e-15 -3.55271368e-15  0.00000000e+00 ...  3.55271368e-15
      0.00000000e+00  0.00000000e+00]
    [ 0.00000000e+00  1.77635684e-15 -1.77635684e-15 ...  1.77635684e-15
      -1.77635684e-15  1.77635684e-15]
    """


In [ ]:
example_1()

Time taken for gergen: 0.031882524490356445
Time taken for numpy: 9.512901306152344e-05
Difference: [[ 0.00000000e+00  0.00000000e+00  3.55271368e-15 ...  0.00000000e+00
   0.00000000e+00 -1.77635684e-15]
 [ 0.00000000e+00 -3.55271368e-15  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.77635684e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.55271368e-15
  -3.55271368e-15 -3.55271368e-15]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -1.77635684e-15]
 [ 3.55271368e-15 -3.55271368e-15  0.00000000e+00 ...  3.55271368e-15
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.77635684e-15 -1.77635684e-15 ...  1.77635684e-15
  -1.77635684e-15  1.77635684e-15]]


**Example 2:**
Using rastgele_gercek(), generate three gergen’s a, b and c with shapes (4,16,16,16). Calculate given equation:

> (a×b + a×c + b×c).ortalama()

Report the time and whether there exists any computational difference in result with their NumPy equivalent.

In [ ]:
def example_2():
    boyut = (4,16,16,16)
    a = rastgele_gercek(boyut)
    b = rastgele_gercek(boyut)
    c = rastgele_gercek(boyut)

    start = time.time()

    birinci = (a*b + a*c + b*c).ortalama()

    end = time.time()


    np_a = np.array(a.listeye())
    np_b = np.array(b.listeye())
    np_c = np.array(c.listeye())

    start_np = time.time()

    ikinci =(np_a*np_b + np_a*np_c + np_b*np_c).mean()

    end_np = time.time()

    print("Time taken for gergen:", end-start)
    print("Time taken for numpy:", end_np-start_np)
    print("Difference:", birinci- ikinci)

    """
    Time taken for gergen: 0.2376084327697754
    Time taken for numpy: 0.000286102294921875
    Difference: 0.0
    """

In [ ]:
example_2()

Time taken for gergen: 0.2376084327697754
Time taken for numpy: 0.000286102294921875
Difference: 0.0


**Example 3**: Using rastgele_gercek(), generate three gergen’s a and b with shapes (3,64,64). Calculate given equation:


> $\frac{\left(\ln\left(\sin(a) + \cos(b)\right)\right)^2}{8}$


Report the time and whether there exists any computational difference in result with their NumPy equivalent.


In [ ]:
def example_3():
    boyut = (3,64,64)
    a = rastgele_gercek(boyut)
    b = rastgele_gercek(boyut)

    start = time.time()

    birinci = ((a.sin() + b.cos()).ln()).us(2) / 8

    end = time.time()

    np_a = np.array(a.listeye())
    np_b = np.array(b.listeye())

    start_np = time.time()

    ikinci = (np.log((np.sin(np_a) + np.cos(np_b))))**2 / 8

    end_np = time.time()

    birinci = np.array(birinci.listeye())

    print("Time taken for gergen:", end-start)
    print("Time taken for numpy:", end_np-start_np)
    print("Difference:", birinci- ikinci)
    """
    Time taken for gergen: 0.17023420333862305
    Time taken for numpy: 0.0005400180816650391
    Difference: [[[0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      ...
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]]

    [[0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      ...
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]]

    [[0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      ...
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]
      [0. 0. 0. ... 0. 0. 0.]]]
    """

In [ ]:
example_3()

Time taken for gergen: 0.17354679107666016
Time taken for numpy: 0.0005543231964111328
Difference: [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
